In [ ]:
import pandas as pd
import os
import ast

In [ ]:
 # Lectura de primer dataset
df_movies = pd.read_csv('movies_dataset.csv')
df_new = df_movies.copy()

In [ ]:
# Desanidar y dividir en belongs_to_collection(id,name)
def desanidar_belongs_to_collection(df, column_name='belongs_to_collection'):
    def desanidar_collection(collection):
        if pd.isna(collection):
            return {"id": None, "name": None}
        try:
            collection_dict = ast.literal_eval(collection)
            if isinstance(collection_dict, dict):
                return {"id": collection_dict.get("id"), "name": collection_dict.get("name")}
            else:
                return {"id": None, "name": None}
        except (ValueError, SyntaxError):
            return {"id": None, "name": None}
    
    # Aplicar la función a la columna y crear nuevas columnas para 'id' y 'name'
    collection_data = df[column_name].apply(desanidar_collection)
    collection_df = pd.json_normalize(collection_data)
    
    # Renombrar columnas para mayor claridad
    collection_df.columns = ['belongs_to_collection_id', 'belongs_to_collection_name']
    
    # Unir las nuevas columnas al dataframe original y eliminar la columna original
    df = pd.concat([df, collection_df], axis=1)
    df = df.drop(columns=[column_name])
    
    return df
df_new = desanidar_belongs_to_collection(df_new,column_name='belongs_to_collection')

In [ ]:
df_new['release_date'] = pd.to_datetime(df_new['release_date'], format = "%Y-%m-%d" , errors='coerce')
df_new["release_date"].dt.year
df_new = df_new.dropna(subset="release_date")

df_new["release_year"] = df_new["release_date"].dt.year
df_new["release_year"] = df_new["release_year"].astype(int)

In [ ]:
def buscar_nombre(lista):
    
    lista = ast.literal_eval(lista)  #en este caso esto convierte los objetos a listas
    
    return ', '.join([i['name'] for i in lista])   # en caso de haber mas de un diccionario con la clave name devuelve todos los valores concatenados

df_new['genres'] = df_new['genres'].fillna('[]').apply(buscar_nombre)
df_new['spoken_languages'] = df_new['spoken_languages'].fillna('[]').apply(buscar_nombre)
df_new['production_countries'] = df_new['production_countries'].fillna('[]').apply(buscar_nombre)
df_new['production_companies'] = df_new['production_companies'].fillna('[]').apply(buscar_nombre)


In [ ]:
df_new[["budget", "revenue"]].fillna(0, inplace = True)
df_new= df_new.dropna(subset=["release_date"])

In [ ]:
df_new["budget"] = df_new["budget"].astype(float)
df_new['return'] = df_new['revenue'] / df_new['budget'].replace(0, 1)
df_new.drop(columns = ["video","imdb_id", "original_title","poster_path" ,"homepage"],inplace = True)
df_new.info()

In [ ]:
def csv_carpeta(df, nombre_archivo, carpeta='Data'):

    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    ruta_archivo = os.path.join(carpeta, nombre_archivo)

    df.to_csv(ruta_archivo, index=False)

In [ ]:
csv_carpeta(df_new[['title','release_date','vote_count','vote_average']],'Datos_de_Pelicula.csv',carpeta='Data')